In [2]:
import os
import numpy as np
from torch.utils.data import DataLoader
import torch
import sys 
sys.path.append('/Users/richardwsong/Desktop/brainhack2025_foundation_models/FoundationModelAdapters/')
from datasets.SampleDataset import SampleDataset, SampleDatasetConfig
import clip 
from torchvision.transforms import ToPILImage

# dataset_config = SampleDatasetConfig()
# dataset = SampleDataset(dataset_config, split_set="test")
# # Create a DataLoader to handle batching
# data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
# print(data_loader)

# print("Reading and testing dataset...")
# for idx, data in enumerate(data_loader):
#     input_eeg_index = data["eeg_index"]
#     input_eeg_spectrograms = data["eeg_spectrogram_img"]
#     input_fmri_spectrograms = data["fmri_spectrogram_imgs"]
    
#     # Print the shape of fMRI spectrograms for each batch
#     print(f"Sample {idx+1}:")
#     print(f"  EEG Index Shape: {input_eeg_index.shape}")
#     print(f"  EEG Spectrograms Shape: {input_eeg_spectrograms.shape}")
#     print(f"  fMRI Spectrograms Shape: {input_fmri_spectrograms.shape}")
#     print("-" * 50)

DATASET_FUNCTIONS = {
    "SampleDataset": [SampleDataset, SampleDatasetConfig],
}

def build_CLIP():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    return model, preprocess

def build_dataset(name):
    dataset_builder = DATASET_FUNCTIONS[name][0]
    dataset_config = DATASET_FUNCTIONS[name][1]()
    dataset_dict = {
        "train": dataset_builder(
            dataset_config,
            split_set="train",
        ),
        "test": dataset_builder(
            dataset_config,
            split_set="test",
        ),
        "zero_shot": dataset_builder(
            dataset_config,
            split_set="zero_shot",
        ),
    }
    return dataset_dict, dataset_config

dataset_dict, dataset_config = build_dataset("SampleDataset")
train, test, zero_shot = dataset_dict["train"], dataset_dict["test"], dataset_dict["zero_shot"]
print(train[0]["fmri_spectrogram_imgs"].shape)
window_size = 30
step_size = 15

all_fmri_spectrograms = []
#for i in range(train[0]["fmri_spectrogram_imgs"].shape[0]):
n = 3
for i in range(n):    
    input_fmri_spectrograms = train[0]["fmri_spectrogram_imgs"][i]
    #print(input_fmri_spectrograms.shape)
    transpose_fmri_spectrograms = input_fmri_spectrograms.transpose(0, 1, 2) 
    # print(transpose_fmri_spectrograms.shape)
    scale_ratio_fmri = transpose_fmri_spectrograms.shape[1] // 690
    transpose_fmri_spectrograms = torch.tensor(transpose_fmri_spectrograms)
    fmri_spectrogram_windows = transpose_fmri_spectrograms.unfold(dimension=1, size=window_size*scale_ratio_fmri, step=step_size*scale_ratio_fmri) # ([4, 114, 570, 3, 30])
    sample_num = 1
    fmri_spectrograms_seq = fmri_spectrogram_windows.reshape(fmri_spectrogram_windows.shape[0], fmri_spectrogram_windows.shape[1], fmri_spectrogram_windows.shape[2], fmri_spectrogram_windows.shape[3]).permute(0, 2, 3, 1)
    # print(fmri_spectrograms_seq.shape)
    # print(len(fmri_spectrograms_seq))

    indices = np.random.permutation(len(fmri_spectrograms_seq))
    fmri_spectrograms = fmri_spectrograms_seq[indices]
    clip_model, clip_preprocess = build_CLIP()
    fmri_spectrogram_windows_preprocessed = torch.stack([clip_preprocess(ToPILImage()(img.byte())) for img in fmri_spectrograms]).to(torch.float32)
    #print(fmri_spectrogram_windows_preprocessed.shape)
    all_fmri_spectrograms.append(fmri_spectrogram_windows_preprocessed)
    print("done with sample", i)

all_fmri_spectrograms = torch.stack(all_fmri_spectrograms, dim=-1)
all_fmri_spectrograms_batch = all_fmri_spectrograms.view(-1, 3, 224, 224)
print(all_fmri_spectrograms_batch.shape)

# fmri_spectrogram_windows_preprocessed = all_fmri_spectrograms_batch.to(clip_model.visual.conv1.weight.device).type_as(clip_model.visual.conv1.weight)
# print("done with this step")
# fmri_clip_feats = clip_model.visual(fmri_spectrogram_windows_preprocessed).to(torch.float32)

# #unbatch 
# fmri_clip_feats = fmri_clip_feats.view(570, 2, 512)
# print(fmri_clip_feats.shape)
# fmri_spectrograms_seq = fmri_spectrogram_windows.reshape(sample_num*fmri_spectrogram_windows.shape[1], fmri_spectrogram_windows.shape[2], fmri_spectrogram_windows.shape[3], fmri_spectrogram_windows.shape[4]).permute(0, 2, 3, 1)


SampleDatasetConfig initialized
Initializing SampleDataset
Done
Initializing SampleDataset
Done
Initializing SampleDataset
Done
data/SampleData/vigall_out/sub01-scan02*.npy
Found fMRI spectrogram files: data/SampleData/eeg_spectrograms/sub01-scan02_eeg_spectrogram.png
Found fMRI spectrogram files: ['data/SampleData/fmri_spectrograms/sub01-scan02_Postcentral gyrus inferior_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan02_Anterior Cingulate Cortex_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan02_Fusiform gyrus_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan02_Cerebellum Crus II_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan02_Precentral gyrus medial_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan02_Lingual gyrus anterior_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan02_Inferior frontal gyrus_fmri_spectrogram.png', 'data/SampleData/fmri_spectrograms/sub01-scan0

In [3]:
fmri_spectrogram_windows_preprocessed = all_fmri_spectrograms_batch.to(clip_model.visual.conv1.weight.device).type_as(clip_model.visual.conv1.weight)
print("done with this step")
print(fmri_spectrogram_windows_preprocessed.shape)
all_feats = []
fmri_clip_feats = clip_model.visual(fmri_spectrogram_windows_preprocessed[0:570, :, :, :]).to(torch.float32)
all_feats.append(fmri_clip_feats)

# #unbatch 
# fmri_clip_feats = fmri_clip_feats.view(570, 2, 512)
# print(fmri_clip_feats.shape)
# fmri_spectrograms_seq = fmri_spectrogram_windows.reshape(sample_num*fmri_spectrogram_windows.shape[1], fmri_spectrogram_windows.shape[2], fmri_spectrogram_windows.shape[3], fmri_spectrogram_windows.shape[4]).permute(0, 2, 3, 1)


done with this step
torch.Size([1710, 3, 224, 224])
